### Learning PyTorch with Examples
* 본질적으로, PyTorch에는 두 가지 주요한 특징이 있음.
  + NumPy와 유사하지만 GPU 상에서 실행 가능한 n-차원 텐서(Tensor)
  + 신경망을 구성하고 학습하는 과정에서의 자동 미분(Automatic differentiation)


* 이 튜토리얼에서는 3차 다항식(third order polynomial)을 사용하여 $ y = sin(x) $ 에 근사(fit)하는 문제를 다뤄보겠음.
* 신경망은 4개의 매개변수를 가지며, 정답과 신경망이 예측한 결과 사이의 유클리드 거리(Euclidean distance)를 최소화하여 임의의 값을 근사할 수 있도록 경사하강법(gradient descent)을 사용하여 학습하겠음.

### Tensors
* Pytorch 를 소개하기 전에, 먼저 NumPy를 사용하여 신경망을 구성해보겠음.

* NumPy는 n-차원 배열 객체와 이러한 배열들을 조작하기 위한 다양한 함수들을 제공함.
* NumPy는 과학 분야의 연산을 위한 포괄적인 프레임워크(generic framework).
* NumPy는 연산 그래프(computation graph)나 딥러닝, 변화도(gradient)에 대해서는 알지 못함.
* 하지만 NumPy 연산을 사용하여 신경망의 순전파 단계와 역전파 단계를 직접 구현함으로써, 3차 다항식이 사인(sine) 함수에 근사하도록 만들 수 있음.

In [1]:
# -*- coding : utf-8 -*-
import numpy as np
import math

# 무작위로 입력과 출력 데이터를 생성함
x = np.linspace(-math.pi, math.pi, 2000)
y = np.sin(x)

# 무작위로 가중치를 초기화함
a = np.random.randn()
b = np.random.randn()
c = np.random.randn()
d = np.random.randn()

learning_rate = 1e-6
for t in range(2000):
    # 순전파 단계 : 예측값 y를 계산함
    # y = a + b x + c x^2 + d x^3
    y_pred = a + b * x + c * x ** 2 + d * x ** 3

    # 손실 (loss)를 계산하고 출력함
    loss = np.square(y_pred - y).sum()
    if t % 100 == 99:
        print(t, loss)

    # 손실에 따른 a, b, c, d의 변화도(gradient)를 계산하고 역전파함
    grad_y_pred = 2.0 * (y_pred - y)
    grad_a = grad_y_pred.sum()
    grad_b = (grad_y_pred * x).sum()
    grad_c = (grad_y_pred * x ** 2).sum()
    grad_d = (grad_y_pred * x ** 3).sum()

    # 가중치를 갱신함
    a -= learning_rate * grad_a
    b -= learning_rate * grad_b
    c -= learning_rate * grad_c
    d -= learning_rate * grad_d

print(f'Result: y = {a} + {b} x + {c} x^2 + {d} x^3')

99 776.5582805896399
199 526.9018247860611
299 358.77147722890703
399 245.44044847389534
499 168.9754285518986
599 117.33384959768229
699 82.42223756241053
799 58.796557119144616
899 42.79167617046276
999 31.937842025484752
1099 24.569248393264303
1199 19.561253106650362
1299 16.15382009479196
1399 13.832793737820761
1499 12.249992751781487
1599 11.169382417491871
1699 10.430780494499139
1799 9.925362065059526
1899 9.579111334333518
1999 9.341630904054682
Result: y = 0.018773728364737213 + 0.8426577425919265 x + -0.0032387805391055915 x^2 + -0.09132719063377252 x^3


### Pytorch : Tensors
* NumPy는 훌륭한 프레임워크지만, GPU를 사용하여 수치 연산을 가속화할 수는 없음.
* 현대의 심층 신경망에서 GPU는 종종 50배 또는 그 이상의 속도 향상을 제공하기 때문에, 안타깝게도 NumPy는 현대의 딥러닝에는 충분치 않음.

* 이번에는 PyTorch의 가장 핵심적인 개념인 **텐서(Tensor)** 에 대해서 알아보겠음.
* PyTorch 텐서(Tensor)는 개념적으로 NumPy 배열과 동일함.
* 텐서(Tensor)는 n-차원 배열이며, PyTorch는 이러한 텐서들의 연산을 위한 다양한 기능들을 제공함.
* NumPy 배열처럼 PyTorch Tensor는 딥러닝이나 연산 그래프, 변화도는 알지 못하며, 과학적 분야의 연산을 위한 포괄적인 도구임.
* 텐서는 연산 그래프와 변화도를 추적할 수도 있지만, 과학적 연산을 위한 일반적인 도구로도 유용함.

* 또한 NumPy와는 다르게, PyTorch 텐서는 GPU를 사용하여 수치 연산을 가속할 수 있음.
* PyTorch 텐서를 GPU에서 실행하기 위해서는 단지 적절한 장치를 지정해주기만 하면 됨.

* 여기에서는 PyTorch 텐서를 사용하여 3차 다항식을 사인(sine) 함수에 근사해보겠음.
* 위의 NumPy 예제에서와 같이 신경망의 순전파 단계와 역전파 단계는 직접 구현하겠음.

In [2]:
import torch

torch.cuda.is_available() # GPU 사용 가능한지 여부 확인

True

In [3]:
# -*- coding : utf-8 -*-
import torch
import math

dtype = torch.float
# device = torch.device("CPU") # CPU 에서 실행하기 위한 코드
device = torch.device("cuda:0") # GPU 에서 실행하기 위한 코드 (나는 GPU로!)

# 무작위로 입력과 출력 데이터를 생성함
x = torch.linspace(-math.pi, math.pi, 2000, device=device, dtype=dtype)
y = torch.sin(x)

# 무작위로 가중치를 초기화함
a = torch.randn((), device=device, dtype=dtype)
b = torch.randn((), device=device, dtype=dtype)
c = torch.randn((), device=device, dtype=dtype)
d = torch.randn((), device=device, dtype=dtype)

learning_rate = 1e-6
for t in range(2000):
    # 순전파 단계 : 예측값 y를 계산함
    y_pred = a + b * x + c * x ** 2 + d * x ** 3

    # 손실 (loss)를 계산하고 출력함
    loss = (y_pred - y).pow(2).sum().item()
    if t % 100 == 99:
        print(t, loss)

    # 손실에 따른 a, b, c, d의 변화도(gradient)를 계산하고 역전파함
    grad_y_pred = 2.0 * (y_pred - y)
    grad_a = grad_y_pred.sum()
    grad_b = (grad_y_pred * x).sum()
    grad_c = (grad_y_pred * x ** 2).sum()
    grad_d = (grad_y_pred * x ** 3).sum()

    # 가중치를 갱신함
    a -= learning_rate * grad_a
    b -= learning_rate * grad_b
    c -= learning_rate * grad_c
    d -= learning_rate * grad_d

print(f'Result: y = {a.item()} + {b.item()} x + {c.item()} x^2 + {d.item()} x^3')

99 937.6567993164062
199 628.136962890625
299 421.96893310546875
399 284.58001708984375
499 192.98141479492188
599 131.88067626953125
699 91.1024398803711
799 63.87183380126953
899 45.67750549316406
999 33.51333999633789
1099 25.37555694580078
1199 19.927776336669922
1299 16.278249740600586
1399 13.831618309020996
1499 12.190160751342773
1599 11.088027954101562
1699 10.347424507141113
1799 9.849325180053711
1899 9.514020919799805
1999 9.28811264038086
Result: y = 0.01316809467971325 + 0.8394556641578674 x + -0.0022717139218002558 x^2 + -0.09087172150611877 x^3


### Autograd
### Pytorch : Tensors and autograd
* 위의 예제들에서는 신경망의 순전파 단계와 역전파 단계를 직접 구현해보았음.
* 작은 2계층 (2-layer) 신경망에서는 역전파 단계를 직접 구현하는 것이 큰일이 아니지만, 복잡한 대규모 신경망에서는 매우 아슬아슬한 일일 것임.

* 다행히도, 자동 미분을 사용하여 신경망의 역전파 단계 연산을 자동화할 수 있음.
* PyTorch의 **autograd** 패키지는 정확히 이런 기능을 제공함.
* Autograd를 사용하면, 신경망의 순전파 단계에서 **연산 그래프(computational graph)**를 정의하게 됨.
* 이 그래프의 노드(node)는 텐서(tensor)이고, 엣지(edge)는 입력 텐서로부터 출력 텐서를 만들어내는 함수가 됨.
* 그래프를 통해 역전파를 하게 되면 변화도를 쉽게 계산할 수 있음.

* 이는 복잡하게 들리겠지만, 실제로 사용하는 것은 매우 간단함.
* 각 텐서는 연산그래프에서 노드로 표현됨.
* 만약 x 가 x.requires_grad=True 인 텐서라면 x.grad 어떤 스칼라 값에 대한 x 의 변화도를 갖는 또 다른 텐서임.

* 여기서는 PyTorch 텐서와 autograd를 사용하여 3차 다항식을 사인파(sine wave)에 근사하는 예제를 구현해보겠음.
* 이제 더 이상 신경망의 역전파 단계를 직접 구현할 필요가 없음.

In [4]:
# -*- coding : utf-8 -*-
import torch
import math

dtype = torch.float
# device = torch.device("cpu")
device = torch.device("cuda:0") # GPU 에서 실행함

# 입력값과 출력값을 갖는 텐서들을 생성함
# requires_grad=False가 기본값으로 설정되어 역전파 단계 중에 이 텐서들에 대한 변화도를
# 계산할 필요가 없음을 나타냄
x = torch.linspace(-math.pi, math.pi, 2000, device=device, dtype=dtype)
y = torch.sin(x)

# 가중치를 갖는 임의의 텐서를 생성함, 3차 다항식이므로 4개의 가중치가 필요함
# y = a + b x + c x^2 + d x^3
# requires_grad=True로 설정하여 역전파 단계 중에 이 텐서들에 대한 변화도를 계산할 필요가
# 있음을 나타냄
a = torch.randn((), device=device, dtype=dtype, requires_grad=True)
b = torch.randn((), device=device, dtype=dtype, requires_grad=True)
c = torch.randn((), device=device, dtype=dtype, requires_grad=True)
d = torch.randn((), device=device, dtype=dtype, requires_grad=True)

learning_rate = 1e-16
for t in range(2000):
    # 순전파 단계 : 텐서들 간의 연산을 사용하여 예측값 y를 계산함
    y_pred = a + b * x + c * x ** 2 + d * x ** 3

    # 텐서들간의 연산을 사용하여 손실(loss)을 계산하고 출력함
    # 이 때 손실은 (1, ) shape을 갖는 텐서임
    # loss.item() 으로 손실이 갖고 있는 스칼라 값을 가져올 수 있음.
    loss = (y_pred - y).pow(2).sum()
    if t % 100 == 99:
        print(t, loss.item())

    # autograd 를 사용하여 역전파 단계를 계산함. 이는 requires_grad=True를 갖는
    # 모든 텐서들에 대한 손실의 변화도를 계산함.
    # 이후 a.grad와 b.grad, c.grad, d.grad는 각각 a, b, c, d에 대한 손실의 변화도를
    # 갖는 텐서가 됨.
    loss.backward()

    # 경사하강법(gradient descent)을 사용하여 가중치를 직접 갱신함.
    # torch.no_grad()로 감싸는 이유는, 가중치들이 requires_grad=True 지만
    # autograd에서는 이를 추적하지 않을 것이기 때문.
    with torch.no_grad():
        a -= learning_rate * a.grad
        b -= learning_rate * b.grad
        c -= learning_rate * c.grad
        d -= learning_rate * d.grad

        # 가중치 갱신 후에는 변화도를 직접 0으로 만듦.
        a.grad = None
        b.grad = None
        c.grad = None
        d.grad = None

print(f'Result: y = {a.item()} + {b.item()} x + {c.item()} x^2 + {d.item()} x^3')

99 248488.015625
199 248488.015625
299 248488.015625
399 248488.015625
499 248488.015625
599 248488.015625
699 248488.015625
799 248488.015625
899 248488.015625
999 248488.015625
1099 248488.015625
1199 248488.015625
1299 248488.015625
1399 248488.015625
1499 248488.015625
1599 248488.015625
1699 248488.015625
1799 248488.015625
1899 248488.015625
1999 248488.015625
Result: y = -0.5007075667381287 + 1.0118036270141602 x + -2.322666883468628 x^2 + -0.3979623317718506 x^3


### PyTorch: Defining new autograd functions
* 내부적으로, autograd의 기본(primitive) 연산자는 실제로 텐서를 조작하는 2개의 함수.
* **forward** 함수는 입력 텐서로부터 출력 텐서를 계산함.
* **backward** 함수는 어떤 스칼라 값에 대한 출력 텐서의 변화도(gradient)를 전달받고, 동일한 스칼라 값에 대한 입력 텐서의 변화도를 계산함.

* PyTorch에서 torch.autograd.Function 의 하위클래스(subclass)를 정의하고 forward 와 backward 함수를 구현함으로써 사용자 정의 autograd 연산자를 손쉽게 정의할 수 있음.
* 그 후, 인스턴스(instance)를 생성하고 이를 함수처럼 호출하고, 입력 데이터를 갖는 텐서를 전달하는 식으로 새로운 autograd 연산자를 사용할 수 있음.

* 이 예제에서는 $ y = a + bx + cx^2 + dx^3 $ 대신 $ y = a + bP_3(c + dx) $ 로 모델을 정의함.
* 여기서 $ P_3(x) = \frac{1}{2} (5x^3 - 3x) $ 은 3차 르장드르 다항식(Legendre polynomial).
* $ P_3 $의 순전파와 역전파 연산을 위한 새로운 autograd Function을 작성하고, 이를 사용하여 모델을 구현함.

In [5]:
# -*- coding : utf-8 -*-
import torch
import math

class LegendrePolynomial3(torch.autograd.Function):
    # torch.autograd.Fucntion 을 상속받아 사용자 정의 autograd Function을 구현하고,
    # 텐서 연산을 하는 순전파 단계와 역전파 단계를 구현해보겠음.
    @staticmethod
    def forward(ctx, input):
        # 순전파 단계에서는 입력을 갖는 텐서를 받아 출력을 갖는 텐서를 반환함.
        # ctx는 컨텍스트 객체(context object)로 역전파 연산을 위한 정보 저장에 사용함.
        # ctx.save_for_backward 메소드를 사용하여 역전파 단계에서 사용할 어떤 객체도
        # 저장(cache)해 둘 수 있음.
        ctx.save_for_backward(input)
        return 0.5 * (5 * input ** 3 - 3 * input)

    @staticmethod
    def backward(ctx, grad_output):
        # 역전파 단계에서는 출력에 대한 손실(loss)의 변화도(gradient)를 갖는 텐서를 받고,
        # 입력에 대한 손실의 변화도를 계산해야 함.
        input, = ctx.saved_tensors
        return grad_output * 1.5 * (5 * input ** 2 - 1)

dtype = torch.float
# device = torch.device("cpu")
device = torch.device("cuda:0") # GPU 에서 실행하기 위함.

# 입력값과 출력값을 갖는 텐서들을 생성함.
# requires_grad=False가 기본값으로 설정되어 역전파 단계 중에 이 텐서들에 대한 변화도를 계산할
# 필요가 없음을 나타냄.
x = torch.linspace(-math.pi, math.pi, 2000, device=device, dtype=dtype)
y = torch.sin(x)

# 가중치를 갖는 임의의 텐서를 생성함.
# 3차 다항식이므로 4개의 가중치가 필요함.
# y = a + b * P3(c + d * x)
# 이 가중치들이 수렴(convergence)하기 위해서는 정답으로부터 너무 멀리 떨어지지 않은 값으로
# 초기화가 되어야 함.
# requires_grad=True로 설정하여 역전파 단계 중에 이 텐서들에 대한 변화도를 계산할 필요가
# 있음을 나타냄.
a = torch.full((), 0.0, device=device, dtype=dtype, requires_grad=True)
b = torch.full((), -1.0, device=device, dtype=dtype, requires_grad=True)
c = torch.full((), 0.0, device=device, dtype=dtype, requires_grad=True)
d = torch.full((), 0.3, device=device, dtype=dtype, requires_grad=True)

learning_rate = 5e-6
for t in range(2000):
    # 사용자 정의 Function을 적용하기 위해 Fucntion.apply 메소드를 사용함.
    # 여기에 'P3'이라고 이름을 붙임.
    P3 = LegendrePolynomial3.apply

    # 순전파 단계 : 연산을 하여 예측값 y를 계산함.
    # 사용자 정의 autograd 연산을 사용하여 P3를 계산함.
    y_pred = a + b * P3(c + d * x)

    # 손실을 계산하고 출력함.
    loss = (y_pred - y).pow(2).sum()
    if t % 100 == 99:
        print(t, loss.item())

    # autograd를 사용하여 역전파 단계를 계산함.
    loss.backward()

    # 경사하강법(gradient descent)을 사용하여 가중치를 갱신함.
    with torch.no_grad():
        a -= learning_rate * a.grad
        b -= learning_rate * b.grad
        c -= learning_rate * c.grad
        d -= learning_rate * d.grad

        # 가중치 갱신 후에는 변화도를 직접 0으로 만듦.
        a.grad = None
        b.grad = None
        c.grad = None
        d.grad = None

print(f'Result: y = {a.item()} + {b.item()} * P3({c.item()} + {d.item()} x)')

99 209.95834350585938
199 144.66018676757812
299 100.70249938964844
399 71.03520202636719
499 50.978515625
599 37.40313720703125
699 28.20686912536621
799 21.973186492919922
899 17.745729446411133
999 14.877889633178711
1099 12.931766510009766
1199 11.610918998718262
1299 10.714248657226562
1399 10.105474472045898
1499 9.692106246948242
1599 9.411375045776367
1699 9.220745086669922
1799 9.091285705566406
1899 9.003361701965332
1999 8.943639755249023
Result: y = -1.765793067320942e-11 + -2.208526849746704 * P3(9.924167737596079e-11 + 0.2554861009120941 x)


### nn Module
### PyTorch: nn

* 연산 그래프와 autograd는 복잡한 연산자를 정의하고 도함수(derivative)를 자동으로 계산하는 매우 강력한 패러다임(paradigm)임.
* 하지만 대규모 신경망에서는 autograd 그 자체만으로는 너무 저수준(low-level)일 수 있음.

* 신경망을 구성하는 것을 종종 연산을 **계층(layer)**에 배열(arrange)하는 것으로 생각하는데, 이 중 일부는 학습 도중 최적화가 될 **학습 가능한 매개변수**를 갖고 있음.

* 텐서플로우(Tensorflow)에서는, Keras와 Tensorflow-Slim, TFLearn 같은 패키지들이 연산 그래프를 고수준(high-level)으로 추상화(abstraction)하여 제공하므로 신경망을 구축하는데 유용함.

* 파이토치(PyTorch)에서는 nn 패키지가 동일한 목적으로 제공됨.
* nn 패키지는 신경망 계층(layer)과 거의 비슷한 **Module** 의 집합을 정의함.
* Module은 입력 텐서를 받고 출력 텐서를 계산하는 한편, 학습 가능한 매개변수를 갖는 텐서들을 내부 상태(internal state)로 가짐.
* nn 패키지는 또한 신경망을 학습시킬 때 주로 사용하는 유용한 손실 함수(loss function)들도 정의하고 있음.

* 이 예제에서는 nn 패키지를 사용하여 다항식 모델을 구현해보겠음.

In [6]:
# -*- coding: utf-8 -*-
import torch
import math

# 입력값과 출력값을 갖는 텐서들을 생성
x = torch.linspace(-math.pi, math.pi, 2000)
y = torch.sin(x)

# 이 예제에서, 출력 y는 (x, x^2, x^3)의 선형 함수이므로, 선형 계층 신경망으로 간주
# (x, x^2, x^3)를 위한 텐서를 준비
p = torch.tensor([1, 2, 3])
xx = x.unsqueeze(-1).pow(p)

# 위 코드에서, x.unsqueeze(-1)은 (2000, 1)의 shape을, p는 (3,)의 shape을 가지므로,
# 이 경우 브로드캐스트(broadcast)가 적용되어 (2000, 3)의 shape을 갖는 텐서를 얻음

# nn 패키지를 사용하여 모델을 순차적 계층(sequence of layers)으로 정의
# nn.Sequential은 다른 Module을 포함하는 Module로, 포함되는 Module들을 순차적으로 적용하여 출력을 생성
# 각각의 Linear Module은 선형 함수(linear function)를 사용하여 입력으로부터
# 출력을 계산하고, 내부 Tensor에 가중치와 편향을 저장
# Flatten 계층은 선형 계층의 출력을 `y` 의 shape과 맞도록(match) 1D 텐서로 핌 (flatten).
model = torch.nn.Sequential(
    torch.nn.Linear(3, 1),
    torch.nn.Flatten(0, 1)
)

# 또한 nn 패키지에는 주로 사용되는 손실 함수(loss function)들에 대한 정의도 포함되어 있음
# 여기에서는 평균 제곱 오차(MSE; Mean Squared Error)를 손실 함수로 사용함
loss_fn = torch.nn.MSELoss(reduction='sum')

learning_rate = 1e-6
for t in range(2000):

    # 순전파 단계: x를 모델에 전달하여 예측값 y를 계산합니다. Module 객체는 __call__ 연산자를 
    # 덮어써서(override) 함수처럼 호출할 수 있도록 합니다. 이렇게 함으로써 입력 데이터의 텐서를 Module에 전달하여
    # 출력 데이터의 텐서를 생성
    y_pred = model(xx)

    # 손실을 계산하고 출력
    # 예측한 y와 정답인 y를 갖는 텐서들을 전달하고,
    # 손실 함수는 손실(loss)을 갖는 텐서를 반환
    loss = loss_fn(y_pred, y)
    if t % 100 == 99:
        print(t, loss.item())

    # 역전파 단계를 실행하기 전에 변화도(gradient)를 0으로 만듦
    model.zero_grad()

    # 역전파 단계: 모델의 학습 가능한 모든 매개변수에 대해 손실의 변화도를 계산함
    # 내부적으로 각 Module의 매개변수는 requires_grad=True일 때 텐서에 저장되므로,
    # 아래 호출은 모델의 모든 학습 가능한 매개변수의 변화도를 계산하게 됨
    loss.backward()

    # 경사하강법을 사용하여 가중치를 갱신
    # 각 매개변수는 텐서이므로, 이전에 했던 것처럼 변화도에 접근할 수 있음
    with torch.no_grad():
        for param in model.parameters():
            param -= learning_rate * param.grad

# list의 첫번째 항목에 접근하는 것처럼 `model` 의 첫번째 계층(layer)에 접근할 수 있음
linear_layer = model[0]

# 선형 계층에서, 매개변수는 `weights` 와 `bias` 로 저장됨
print(f'Result: y = {linear_layer.bias.item()} + {linear_layer.weight[:, 0].item()} x + {linear_layer.weight[:, 1].item()} x^2 + {linear_layer.weight[:, 2].item()} x^3')

99 721.7434692382812
199 483.88482666015625
299 325.5351257324219
399 220.07228088378906
499 149.8014373779297
599 102.95714569091797
699 71.71416473388672
799 50.865570068359375
899 36.94554138183594
999 27.64630126953125
1099 21.430177688598633
1199 17.272315979003906
1299 14.48936653137207
1399 12.62541389465332
1499 11.376047134399414
1599 10.538021087646484
1699 9.9754638671875
1799 9.59749984741211
1899 9.343368530273438
1999 9.172338485717773
Result: y = -0.011105677112936974 + 0.8415225744247437 x + 0.001915913773700595 x^2 + -0.09116571396589279 x^3


### PyTorch: optim
* 지금까지는 torch.no_grad() 로 학습 가능한 매개변수들을 갖는 텐서들을 직접 조작하여 모델의 가중치(weight)를 갱신함.
* 이것은 확률적 경사하강법(SGD: Stochastic Gradient Descent)와 같은 간단한 최적화 알고리즘에서는 크게 부담이 되진 않지만, 실제로 신경망을 학습할 때는 AdaGrad, RMSProp, Adam 등과 같은 더 정교한 옵티마이저(optimizer)를 사용하곤 함.

* PyTorch의 optim 패키지는 최적화 알고리즘에 대한 아이디어를 추상화하고 일반적으로 사용하는 최적화 알고리즘의 구현체(implementation)를 제공함.

* 이 예제에서는 지금까지와 같이 nn 패키지를 사용하여 모델을 정의하지만, 모델을 최적화할 때는 optim 패키지가 제공하는 RMSProp 알고리즘을 사용함.

In [7]:
# -*- coding: utf-8 -*-
import torch
import math

# 입력값과 출력값을 갖는 텐서들을 생성
x = torch.linspace(-math.pi, math.pi, 2000)
y = torch.sin(x)

# 입력 텐서 (x, x^2, x^3)를 준비
p = torch.tensor([1, 2, 3])
xx = x.unsqueeze(-1).pow(p)

# nn 패키지를 사용하여 모델과 손실 함수를 정의
model = torch.nn.Sequential(
    torch.nn.Linear(3, 1),
    torch.nn.Flatten(0, 1)
)
loss_fn = torch.nn.MSELoss(reduction='sum')

# optim 패키지를 사용하여 모델의 가중치를 갱신할 optimizer를 정의
# 여기서는 RMSprop을 사용
# optim 패키지는 다른 다양한 최적화 알고리즘을 포함하고 있음
# RMSprop 생성자의 첫번째 인자는 어떤 텐서가 갱신되어야 하는지를 알려줌
learning_rate = 1e-3
optimizer = torch.optim.RMSprop(model.parameters(), lr=learning_rate)
for t in range(2000):
    # 순전파 단계: 모델에 x를 전달하여 예측값 y를 계산
    y_pred = model(xx)

    # 손실을 계산하고 출력합니다.
    loss = loss_fn(y_pred, y)
    if t % 100 == 99:
        print(t, loss.item())

    # 역전파 단계 전에, optimizer 객체를 사용하여 (모델의 학습 가능한 가중치인) 갱신할
    # 변수들에 대한 모든 변화도(gradient)를 0으로 만듦
    # 이렇게 하는 이유는 기본적으로 
    # .backward()를 호출할 때마다 변화도가 버퍼(buffer)에 (덮어쓰지 않고) 누적되기 때문
    optimizer.zero_grad()

    # 역전파 단계: 모델의 매개변수들에 대한 손실의 변화도를 계산
    loss.backward()

    # optimizer의 step 함수를 호출하면 매개변수가 갱신
    optimizer.step()


linear_layer = model[0]
print(f'Result: y = {linear_layer.bias.item()} + {linear_layer.weight[:, 0].item()} x + {linear_layer.weight[:, 1].item()} x^2 + {linear_layer.weight[:, 2].item()} x^3')

99 3402.990234375
199 1438.7095947265625
299 673.4249267578125
399 365.02899169921875
499 237.97328186035156
599 152.7903289794922
699 87.22942352294922
799 43.701175689697266
899 20.209787368774414
999 11.027052879333496
1099 8.988139152526855
1199 9.138055801391602
1299 8.827688217163086
1399 8.849699974060059
1499 8.940855026245117
1599 8.95499038696289
1699 8.895038604736328
1799 8.89332103729248
1899 8.91107177734375
1999 8.915462493896484
Result: y = -0.00019108649576082826 + 0.857255220413208 x + -0.00019188056467100978 x^2 + -0.09281580150127411 x^3


### PyTorch: Custom nn Modules

* 때때로 기존 Module의 구성(sequence)보다 더 복잡한 모델을 구성해야 할때가 있음
* 이러한 경우에는 nn.Module 의 하위 클래스(subclass)로 새로운 Module 을 정의하고, 입력 텐서를 받아 다른 모듈 및 autograd 연산을 사용하여 출력 텐서를 만드는 forward 를 정의함.

* 이 예제에서는 3차 다항식을 사용자 정의 Module 하위 클래스(subclass)로 구현해보겠음.

In [8]:
# -*- coding: utf-8 -*-
import torch
import math


class Polynomial3(torch.nn.Module):
    def __init__(self):
        """
        생성자에서 4개의 매개변수를 생성(instantiate)하고, 멤버 변수로 지정
        """
        super().__init__()
        self.a = torch.nn.Parameter(torch.randn(()))
        self.b = torch.nn.Parameter(torch.randn(()))
        self.c = torch.nn.Parameter(torch.randn(()))
        self.d = torch.nn.Parameter(torch.randn(()))

    def forward(self, x):
        """
        순전파 함수에서는 입력 데이터의 텐서를 받고 출력 데이터의 텐서를 반환해야 함
        텐서들 간의 임의의 연산뿐만 아니라, 생성자에서 정의한 Module을 사용할 수 있음
        """
        return self.a + self.b * x + self.c * x ** 2 + self.d * x ** 3

    def string(self):
        """
        Python의 다른 클래스(class)처럼, PyTorch 모듈을 사용해서 사용자 정의 메소드를 정의할 수 있음
        """
        return f'y = {self.a.item()} + {self.b.item()} x + {self.c.item()} x^2 + {self.d.item()} x^3'


# 입력값과 출력값을 갖는 텐서들을 생성
x = torch.linspace(-math.pi, math.pi, 2000)
y = torch.sin(x)

# 위에서 정의한 클래스로 모델을 생성
model = Polynomial3()

# 손실 함수와 optimizer를 생성
# SGD 생성자에 model.paramaters()를 호출해주면
# 모델의 멤버 학습 가능한 (torch.nn.Parameter로 정의된) 매개변수들이 포함됨
criterion = torch.nn.MSELoss(reduction='sum')
optimizer = torch.optim.SGD(model.parameters(), lr=1e-6)
for t in range(2000):
    # 순전파 단계: 모델에 x를 전달하여 예측값 y를 계산
    y_pred = model(x)

    # 손실을 계산하고 출력
    loss = criterion(y_pred, y)
    if t % 100 == 99:
        print(t, loss.item())

    # 변화도를 0으로 만들고, 역전파 단계를 수행하고, 가중치를 갱신
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

print(f'Result: {model.string()}')

99 3297.433837890625
199 2219.85400390625
299 1496.6529541015625
399 1010.8972778320312
499 684.3565063476562
599 464.6562805175781
699 316.7076721191406
799 216.9860382080078
899 149.7073211669922
999 104.27252197265625
1099 73.55868530273438
1199 52.77511978149414
1299 38.696510314941406
1399 29.149606704711914
1499 22.66864776611328
1599 18.264225006103516
1699 15.267695426940918
1799 13.226666450500488
1899 11.834925651550293
1999 10.884824752807617
Result: y = -0.035115379840135574 + 0.8265110850334167 x + 0.006057987455278635 x^2 + -0.08903046697378159 x^3


### PyTorch: Control Flow + Weight Sharing
* 동적 그래프와 가중치 공유의 예를 보이기 위해, 매우 이상한 모델을 구현해보겠음.
* 각 순전파 단계에서 3 ~ 5 사이의 임의의 숫자(random number)를 선택하여 다차항들에서 사용하고, 동일한 가중치를 여러번 재사용하여 4차항과 5차항을 계산함.

* 이 모델에서는 일반적인 Python 제어 흐름을 사용하여 반복(loop)을 구현할 수 있으며, 순전파 단계를 정의할 때 동일한 매개변수를 여러번 재사용하여 가중치 공유를 구현할 수 있음.

* 이러한 모델을 Module을 상속받는 하위클래스로 간단히 구현해보겠음.

In [9]:
# -*- coding: utf-8 -*-
import random
import torch
import math


class DynamicNet(torch.nn.Module):
    def __init__(self):
        """
        생성자에서 5개의 매개변수를 생성(instantiate)하고 멤버 변수로 지정
        """
        super().__init__()
        self.a = torch.nn.Parameter(torch.randn(()))
        self.b = torch.nn.Parameter(torch.randn(()))
        self.c = torch.nn.Parameter(torch.randn(()))
        self.d = torch.nn.Parameter(torch.randn(()))
        self.e = torch.nn.Parameter(torch.randn(()))

    def forward(self, x):
        """
        모델의 순전파 단계에서는 무작위로 4, 5 중 하나를 선택한 뒤 매개변수 e를 재사용하여
        이 차수들의의 기여도(contribution)를 계산

        각 순전파 단계는 동적 연산 그래프를 구성하기 때문에, 모델의 순전파 단계를 정의할 때
        반복문이나 조건문과 같은 일반적인 Python 제어-흐름 연산자를 사용할 수 있음

        여기에서 연산 그래프를 정의할 때 동일한 매개변수를 여러번 사용하는 것이 완벽히 안전하다는
        것을 알 수 있음
        """
        y = self.a + self.b * x + self.c * x ** 2 + self.d * x ** 3
        for exp in range(4, random.randint(4, 6)):
            y = y + self.e * x ** exp
        return y

    def string(self):
        """
        Python의 다른 클래스(class)처럼, PyTorch 모듈을 사용해서 사용자 정의 메소드를 정의할 수 있음
        """
        return f'y = {self.a.item()} + {self.b.item()} x + {self.c.item()} x^2 + {self.d.item()} x^3 + {self.e.item()} x^4 ? + {self.e.item()} x^5 ?'


# 입력값과 출력값을 갖는 텐서들을 생성
x = torch.linspace(-math.pi, math.pi, 2000)
y = torch.sin(x)

# 위에서 정의한 클래스로 모델을 생성
model = DynamicNet()

# 손실 함수와 optimizer를 생성
# 이 이상한 모델을 순수한 확률적 경사하강법(SGD; Stochastic Gradient Descent)으로
# 학습하는 것은 어려우므로, 모멘텀(momentum)을 사용합니다.
criterion = torch.nn.MSELoss(reduction='sum')
optimizer = torch.optim.SGD(model.parameters(), lr=1e-8, momentum=0.9)
for t in range(30000):
    # 순전파 단계: 모델에 x를 전달하여 예측값 y를 계산
    y_pred = model(x)

    # 손실을 계산하고 출력합니다.
    loss = criterion(y_pred, y)
    if t % 2000 == 1999:
        print(t, loss.item())

    # 변화도를 0으로 만들고, 역전파 단계를 수행하고, 가중치를 갱신
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

print(f'Result: {model.string()}')

1999 61.92776107788086
3999 34.54628372192383
5999 19.4853572845459
7999 13.573152542114258
9999 11.063141822814941
11999 9.801002502441406
13999 9.059605598449707
15999 8.835355758666992
17999 8.945249557495117
19999 8.870192527770996
21999 8.85826301574707
23999 8.630354881286621
25999 8.825692176818848
27999 8.834794998168945
29999 8.859306335449219
Result: y = 0.0013924038503319025 + 0.8570119142532349 x + -0.0007548669818788767 x^2 + -0.09372647106647491 x^3 + 0.0001253405207535252 x^4 ? + 0.0001253405207535252 x^5 ?
